# Librairies

In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import datetime

# Import des données

In [2]:
df_ventes_original = pd.read_csv('./raw/ventes.csv')
df_produits_original = pd.read_csv('./raw/produits.csv')
df_clients_original = pd.read_csv('./raw/clients.csv')
df_ventes, df_produits, df_clients = df_ventes_original, df_produits_original, df_clients_original

In [3]:
df_ventes.info()
df_ventes.describe()
df_ventes.head(30)
df_ventes.isna().sum().sum()
print(df_ventes.nunique())
dups_shape = df_ventes.pivot_table(columns=['session_id'], aggfunc='size')
dups_shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337016 entries, 0 to 337015
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id_prod     337016 non-null  object
 1   date        337016 non-null  object
 2   session_id  337016 non-null  object
 3   client_id   337016 non-null  object
dtypes: object(4)
memory usage: 10.3+ MB
id_prod         3266
date          336855
session_id    169195
client_id       8602
dtype: int64


session_id
s_0        200
s_1          1
s_10         1
s_100        2
s_1000       4
          ... 
s_99994      3
s_99995      1
s_99996      4
s_99997      1
s_99998      3
Length: 169195, dtype: int64

In [4]:
df_produits.info()
df_produits.describe()
df_produits.head(30)
df_produits.isna().sum().sum()
df_produits.nunique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3287 entries, 0 to 3286
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id_prod  3287 non-null   object 
 1   price    3287 non-null   float64
 2   categ    3287 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 77.2+ KB


id_prod    3287
price      1455
categ         3
dtype: int64

In [5]:
df_clients.info()
df_clients.describe()
df_clients.head(30)
df_clients.isna().sum().sum()
dups_shape = df_clients.pivot_table(columns=['client_id'], aggfunc='size')
print (dups_shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8623 entries, 0 to 8622
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   client_id  8623 non-null   object
 1   sex        8623 non-null   object
 2   birth      8623 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 202.2+ KB
client_id
c_1       1
c_10      1
c_100     1
c_1000    1
c_1001    1
         ..
c_997     1
c_998     1
c_999     1
ct_0      1
ct_1      1
Length: 8623, dtype: int64


# Merge des dataframes 

In [6]:
# df_ventes, df_produits, df_clients
df_temp = pd.merge(df_ventes,
                        df_clients,left_on='client_id', right_on='client_id', how="inner")
df_full =  pd.merge(df_temp,
                        df_produits, left_on='id_prod', right_on='id_prod', how="inner")
df_full.sort_values('date', ascending = False, inplace = True)

In [7]:
df_full.info()
df_full.describe()
df_full.head(30)
df_full.isna().sum().sum()
df_full.head(202)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336913 entries, 335980 to 240413
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id_prod     336913 non-null  object 
 1   date        336913 non-null  object 
 2   session_id  336913 non-null  object 
 3   client_id   336913 non-null  object 
 4   sex         336913 non-null  object 
 5   birth       336913 non-null  int64  
 6   price       336913 non-null  float64
 7   categ       336913 non-null  int64  
dtypes: float64(1), int64(2), object(5)
memory usage: 23.1+ MB


,id_prod,date,session_id,client_id,sex,birth,price,categ
335980,T_0,test_2021-03-01 02:30:02.237450,s_0,ct_0,f,2001,-1.00,0
336033,T_0,test_2021-03-01 02:30:02.237449,s_0,ct_0,f,2001,-1.00,0
335944,T_0,test_2021-03-01 02:30:02.237449,s_0,ct_1,m,2001,-1.00,0
336032,T_0,test_2021-03-01 02:30:02.237449,s_0,ct_0,f,2001,-1.00,0
336070,T_0,test_2021-03-01 02:30:02.237448,s_0,ct_0,f,2001,-1.00,0
...,...,...,...,...,...,...,...,...
335887,T_0,test_2021-03-01 02:30:02.237412,s_0,ct_1,m,2001,-1.00,0
335882,T_0,test_2021-03-01 02:30:02.237412,s_0,ct_1,m,2001,-1.00,0
335906,T_0,test_2021-03-01 02:30:02.237412,s_0,ct_1,m,2001,-1.00,0
276460,0_1775,2022-02-28 23:59:58.040472,s_172423,c_1460,m,1989,6.99,0


## Il y a un prix negatif pour l'id prod = T_0 : prix = -1
## De plus cela correspond à 200 achats, des dates commençant par test_ et les clients ct_0 et ct_1
## cela ressemble à des données de test on les supprimera pour l'analyse


In [8]:
df_full= df_full.drop(df_full[df_full.price < 0 ].index)
df_full.head(202)

,id_prod,date,session_id,client_id,sex,birth,price,categ
276460,0_1775,2022-02-28 23:59:58.040472,s_172423,c_1460,m,1989,6.99,0
224665,0_1403,2022-02-28 23:59:02.381670,s_172423,c_1460,m,1989,15.99,0
190844,0_1538,2022-02-28 23:57:12.741573,s_172424,c_1876,m,1983,8.61,0
40284,1_370,2022-02-28 23:56:57.561424,s_172423,c_1460,m,1989,13.11,1
128446,1_456,2022-02-28 23:56:57.196281,s_172422,c_5787,f,1986,28.27,1
...,...,...,...,...,...,...,...,...
52582,1_445,2022-02-28 19:22:34.048975,s_172316,c_8261,m,1938,23.99,1
85861,1_259,2022-02-28 19:20:40.130507,s_172322,c_466,m,1940,10.99,1
75695,0_2030,2022-02-28 19:19:44.981612,s_172327,c_3357,m,1972,13.40,0
65134,0_2296,2022-02-28 19:19:14.276720,s_172318,c_5875,f,1979,16.95,0


In [9]:
print(df_full.date.nunique())
df_full.shape

336713


(336713, 8)

In [10]:
df_full['date'] = pd.to_datetime(df_full['date'], format="%Y-%m-%d %H:%M:%S.%f")
df_full.describe()
type(df_full['date'][2])

pandas._libs.tslibs.timestamps.Timestamp

In [11]:
print(datetime.date.today().year)
df_full ['consumer_age'] = datetime.date.today().year - df_full['birth']
df_full

2021


,id_prod,date,session_id,client_id,sex,birth,price,categ,consumer_age
276460,0_1775,2022-02-28 23:59:58.040472,s_172423,c_1460,m,1989,6.99,0,32
224665,0_1403,2022-02-28 23:59:02.381670,s_172423,c_1460,m,1989,15.99,0,32
190844,0_1538,2022-02-28 23:57:12.741573,s_172424,c_1876,m,1983,8.61,0,38
40284,1_370,2022-02-28 23:56:57.561424,s_172423,c_1460,m,1989,13.11,1,32
128446,1_456,2022-02-28 23:56:57.196281,s_172422,c_5787,f,1986,28.27,1,35
...,...,...,...,...,...,...,...,...,...
112464,0_1358,2021-03-01 00:05:18.801198,s_5,c_2033,f,1956,16.49,0,65
43380,0_1458,2021-03-01 00:04:54.559692,s_4,c_7912,f,1989,6.55,0,32
167345,0_1352,2021-03-01 00:02:38.311413,s_3,c_580,m,1988,4.50,0,33
236663,0_1390,2021-03-01 00:02:26.047414,s_2,c_664,m,1960,19.37,0,61


In [12]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336713 entries, 276460 to 240413
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   id_prod       336713 non-null  object        
 1   date          336713 non-null  datetime64[ns]
 2   session_id    336713 non-null  object        
 3   client_id     336713 non-null  object        
 4   sex           336713 non-null  object        
 5   birth         336713 non-null  int64         
 6   price         336713 non-null  float64       
 7   categ         336713 non-null  int64         
 8   consumer_age  336713 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 33.8+ MB


In [13]:
(df_full['sex']=='m').sum()

169198

# Récuperons les données pour l'analyse

In [14]:
if not os.path.exists('curated'):
    os.mkdir('curated')
df_full.to_csv (r'./curated/df_full_curated.csv', index = False, header=True)

# Recuperons la liste des articles non vendus : 

En effect les articles qui ne sont jamais vendus meritent d'etre listés et remontés au manager dans le but eventuel de les retirer de la vente

In [15]:
df_non_vendu = df_ventes.merge(df_produits, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='right_only']
df_non_vendu.to_csv (r'./curated/df_non_vendu.csv', index = False, header=True)


On recense 22 articles jamais vendu

In [16]:
pd.to_datetime('2022-02-28 23:56:57.196281', format="%Y-%m-%d %H:%M:%S.%f")

Timestamp('2022-02-28 23:56:57.196281')